<a href="https://colab.research.google.com/github/ArmHPC/Computational-Astrophysics/blob/main/Cloud_Service/infer_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone Repository

In [ ]:
!git clone https://github.com/ArmHPC/Computational-Astrophysics.git

## Setup Environment

In [ ]:
mv ./Computational-Astrophysics/ ./Project/

In [ ]:
cd Project

In [ ]:
mv ./Cloud_Service/ ./Model/

In [ ]:
cd Model

In [ ]:
# !git pull

## Additional Environment Setups

In [ ]:
import zipfile
with zipfile.ZipFile('./data_Inference.zip', 'r') as zip_ref:
    zip_ref.extractall('')

with zipfile.ZipFile('./Checkpoint.zip', 'r') as zip_ref:
    zip_ref.extractall('')

## Imports

In [ ]:
import os

import torch
from torchsummary import summary

import numpy as np

# from Model.test import evaluate
from Project.Model.load_data import load_images
from Project.Model.models import Model
from Project.Model.infer import infer_evaluate

# from sklearn.metrics import classification_report

## Environment variables

In [ ]:
use_gpu = True

In [ ]:
# Training Device
if use_gpu:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = torch.device('cpu')

print(f'Device: {device}')

## Data path

In [ ]:
# Choose the dataset
num_classes = 10 # Choices: {5, 6, 10}

In [ ]:
classes_5 = ['C-H', 'C-N', 'Mrk SB', 'sdA', 'sdB']
classes_6 = ['C-H', 'C-N', 'Mrk Abs', 'Mrk SB', 'sdA', 'sdB']
classes_10 = ['C Ba', 'C-H', 'C-N', 'C-R', 'Mrk Abs', 'Mrk AGN', 'Mrk SB', 'sdA', 'sdB', 'sdO']

assert num_classes in {5, 6, 10}

if num_classes == 10:
    classes = classes_10
    checkpoint_name = 'Dense_10_Focal_25_3_Final/139.pth'
elif num_classes == 5:
    classes = classes_5
    checkpoint_name = 'Dense_5_High_Focal_25_3_Final/59.pth'
else:
    classes = classes_6
    checkpoint_name = 'Dense_6_High_Focal_25_3_Final/136.pth'


In [ ]:
# Datasets
data_root = f'./data_Inference'

In [ ]:
infer_dir = f'{data_root}/Subtypes/19_2'
input_shape = (160, 50)

In [ ]:
print('Num classes:', num_classes)

## Project Parameters

In [ ]:
root_dir = os.path.abspath('./')

In [ ]:
# Checkpoints are saved in Checkpoint folder
checkpoint_path = f'{root_dir}/Checkpoint/{checkpoint_name}'

print('Selected checkpoint:', checkpoint_path)

## Testing Parameters

In [ ]:
# Batch sizes
infer_batch_size = 256

## Data Loaders

In [ ]:
infer_data = load_images(
    path=infer_dir, batch_size=infer_batch_size,
    domain='inference', _drop_last=False
)

## Training Setup

In [ ]:
# Model choices: arch = any(['default', 'default_prev', 'default_bn', 'mobilenet', 'resnet'])
#   default: the proposed network
#   default_bn: similar to the proposed, but with more BatchNorm layers
#   default_prev: the network proposed in the previous work
#   mobilenet: MobileNetV2
#   resnet: Resnet

net = Model(num_classes=num_classes, input_shape=input_shape, arch='default').to(device)

### Layers

In [ ]:
print(net)

### Output Summary

In [ ]:
summary(net, (1, 160, 50))

In [ ]:
# Setting the network up for evaluation
if device == torch.device('cpu'):
    net.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))
else:
    net.load_state_dict(torch.load(checkpoint_path))
net.eval()

print('Loaded checkpoint:', checkpoint_path)

In [ ]:
print('\nEvaluation started:')

predictions = infer_evaluate(dataloader=infer_data, model=net, device=device)

In [ ]:
len(predictions)

In [ ]:
np.unique(predictions, return_counts=True)